<div style="text-align: center; background-color: #0A6EBD; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 30px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  LAB-02: LÀM VIỆC VÀ TRỰC QUAN HÓA DỮ LIỆU CHUỖI THỜI GIAN<Title>
</div>

<h2 style="text-align:center;">Nhóm 23</h2>

## Thông tin nhóm: 

<center>

| HỌ TÊN                | MSSV      |
|:------------------    |:--------: |
| Lê Công Đắt     | 20120454  |
| Lê Trần Minh Khuê     | 21120279  |
| Hoàng Trung Nam       | 21120290  |
| Lê Trần Như Ngọc       | 21120296  |
| Lê Thị Minh Phương | 21120533  |

</center>

## Giảng viên hướng dẫn:
**Thầy Lê Nhựt Nam**

### Chủ đề: **Phân tích và trực quan giá cổ phiếu của top 6 ngân hàng có giá trị thương hiệu tốt nhất Việt Nam**

## **Phần B: Khám phá và tiền xử lý dữ liệu.**

### **I. Chuẩn bị**

#### 1. Nhập các thư viện

In [1]:
# Nhập các thư viện cần dùng
import numpy as np
import pandas as pd
import os

#### 2. Nhập các file dữ liệu vào notebook

Vì dữ liệu được lưu trữ theo từng ngân hàng riêng biệt, nên ta sẽ đọc từng file lại, tiền xử lý và sau đó gộp lại 1 file sau cùng.

In [13]:
path = "./Data/"
stock_symbol = ['VCB','BID','CTG','TCB','AGR','VPB']
df =[]
for bank in stock_symbol:
    raw_df = pd.read_csv(path + bank +' Historical Data.csv')
    df.append(raw_df)

VCB_df = df[0]
BID_df = df[1]
CTG_df = df[2]
TCB_df = df[3]
AGR_df = df[4]
VPB_df = df[5]

### **II. Khám phá dữ liệu**

#### 1. Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Ta sẽ in ra dòng đầu của từng ngân hàng để xem xét.

In [14]:
VCB_df.head(1)

,Date,Price,Open,High,Low,Vol.,Change %
0,05/24/2024,"90,200","91,000","91,000","90,000",2.24M,-0.88%


In [15]:
BID_df.head(1)

,Date,Price,Open,High,Low,Vol.,Change %
0,05/24/2024,"49,200.0","49,600.0","50,600.0","48,500.0",2.18M,-0.61%


In [16]:
CTG_df.head(1)

,Date,Price,Open,High,Low,Vol.,Change %
0,05/24/2024,"32,500.0","33,100.0","33,900.0","32,300.0",13.92M,-2.26%


In [17]:
TCB_df.head(1)

,Date,Price,Open,High,Low,Vol.,Change %
0,05/24/2024,"46,400.0","46,150.0","47,250.0","45,800.0",14.62M,-0.22%


In [18]:
AGR_df.head(1)

,Date,Price,Open,High,Low,Vol.,Change %
0,05/24/2024,"19,650.0","20,450.0","20,450.0","19,200.0",3.62M,-4.38%


In [21]:
VPB_df.head(1)

,Date,Price,Open,High,Low,Vol.,Change %
0,05/24/2024,"18,000.0","18,400.0","18,500.0","18,000.0",19.56M,-2.70%
1,05/23/2024,"18,500.0","18,300.0","18,500.0","18,150.0",9.37M,1.09%


Ta thấy, mỗi dòng dữ liệu đều thể hiện thông tin về giá chứng khoán của ngân hàng trong một ngày cụ thể. Không có sự khác biệt trong ý nghĩa của các dòng.
Tuy nhiên, cần xem xét thêm có sự trùng lặp nào ở các dòng hay không.

In [23]:
def is_duplicate_rows(df,symbol):
    if len(df[df.duplicated()]) == 0:
        print(f'There are no duplicate rows in {symbol}_df')
    else:
        print('There are duplicate rows in VCB_df')

is_duplicate_rows(VCB_df,'VCB')
is_duplicate_rows(BID_df,'BID')
is_duplicate_rows(CTG_df,'CTG')
is_duplicate_rows(TCB_df,'TCB')
is_duplicate_rows(AGR_df,'AGR')
is_duplicate_rows(VPB_df,'VPB')


There are no duplicate rows in VCB_df
There are no duplicate rows in BID_df
There are no duplicate rows in CTG_df
There are no duplicate rows in TCB_df
There are no duplicate rows in AGR_df
There are no duplicate rows in VPB_df


#### 2. Mỗi cột có ý nghĩa gì?

Các cột và thông tin tương ứng:
- `Date`: Ngày ghi nhận thông tin (đóng vai trò là index khi nằm ở các dataframe của từng ngân hàng).

- `Price`: Giá tham chiếu của ngày đang đề cập, tạm xem như giá đóng cửa (Close price - giá trung bình của các lệnh khớp trong 15p cuối phiên, thường là 14h30-14h45p).

- `Open`: Giá phiên khớp lệnh mở cửa (Được ghi nhận sau phiên khớp lệnh đầu ngày từ 9h00 - 9h15 các ngày thứ 2 đến thứ 6).

- `High`: Giá cao nhất trong phiên giao dịch của ngày đang đề cập.

- `Low`: Giá thấp nhất trong phiên giao dịch của ngày đang đề cập.

- `Vol.`: Khối lượng giao dịch - Tổng số cổ phiếu được mua và bán trong một phiên giao dịch

- `Change %`: Biến động giá - sự thay đổi giá cổ phiếu so với giá tham chiếu (trong ngữ cảnh này thể hiện sự thay đổi của Price ngày hiện tại so với price ngày liền trước.)

#### 3. Mỗi cột hiện có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp cần xử lý tiếp hay không?

In [27]:
print("VCB info"VCB_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3718 entries, 0 to 3717
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      3718 non-null   object
 1   Price     3718 non-null   object
 2   Open      3718 non-null   object
 3   High      3718 non-null   object
 4   Low       3718 non-null   object
 5   Vol.      3718 non-null   object
 6   Change %  3718 non-null   object
dtypes: object(7)
memory usage: 203.5+ KB
None


#### 4. Với mỗi cột, các giá trị (dạng số, dạng phân loại) được phân bố như thế nào?

##### 4.1. Các cột dạng số

##### 4.2. Các cột dạng phân loại

#### 5. Có cần phải tiền xử lý dữ liệu hay không và nếu có thì nhóm cần phải xử lý như thế nào?
